# Clustering with Apach Spark on Elastic MapReduce(EMR)

#### Data Set Information:

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.


#### Attribute Information:

- **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- **Description**: Product (item) name. Nominal.
- **Quantity**: The quantities of each product (item) per transaction. Numeric.
- **InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.
- **UnitPrice**: Unit price. Numeric, Product price per unit in sterling.
- **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- **Country**: Country name. Nominal, the name of the country where each customer resides.


**Download the Dataset**:
[Online Retail Data Set](https://archive.ics.uci.edu/ml/datasets/online+retail)

#### Starting Spark application and loading the dataset

In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SQLContext

from pyspark.sql.functions import *


sc = SparkContext('local', 'Pred_clicks')
sql = SQLContext(sc)

In [2]:
#load the dataframe and inspect the dataset

#run on aws
#from os.path import expanduser

#SRC_PATH = expanduser("~") + '/SageMaker/My-Bucket-S3/'
#df = spark.read.csv('data.csv', header=True, inferSchema=True)

df = sql.read.csv('data.csv', header=True, inferSchema=True)
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [3]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
print("Shape: ({},{})".format(df.count(), len(df.columns)))

Shape: (541909,8)


The dataset involves transactions of products bought by different customers at different times and in different locations. We attempt to cluster these customer transactions using **k-means** by looking at three factors:
- The product (represented by **Stockcode** column)
- The **Country** where the product was bought
- The total amount spent by the customer across al products

In [7]:
df = df.selectExpr("*", "Quantity * UnitPrice as TotalBought")

In [9]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|       TotalBought|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|15.299999999999999|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|              22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|             20.34|
+---------+---------+---

In [11]:
customer_df = df.select("CustomerID",
                        "TotalBought").groupBy("CustomerID").sum(
                        "TotalBought").withColumnRenamed("sum(TotalBought)", "SumTotalBought")
customer_df.show(5)

+----------+------------------+
|CustomerID|    SumTotalBought|
+----------+------------------+
|     17420| 598.8300000000002|
|     16861|            151.65|
|     16503|1421.4300000000005|
|     15727| 5178.960000000001|
|     17389|31300.080000000016|
+----------+------------------+
only showing top 5 rows



In [12]:
joined_df = df.join(customer_df, 'CustomerId')
joined_df.show(5)

+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country|       TotalBought|   SumTotalBought|
+----------+---------+---------+--------------------+--------+--------------+---------+--------------+------------------+-----------------+
|     17850|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|United Kingdom|15.299999999999999|5288.630000000009|
|     17850|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|United Kingdom|             20.34|5288.630000000009|
|     17850|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|United Kingdom|              22.0|5288.630000000009|
|     17850|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|United Kingdom|             20.34|5288.630000000009|
|     17850|   53636

In [13]:
joined_df.limit(5).toPandas()

,CustomerID,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalBought,SumTotalBought
0,17850,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,United Kingdom,15.30,5288.63
1,17850,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63
2,17850,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,United Kingdom,22.00,5288.63
3,17850,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63
4,17850,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63


In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler


stages = [StringIndexer(inputCol = "StockCode", outputCol = "stock_code_index", handleInvalid = 'keep'),
          OneHotEncoder(inputCol= "stock_code_index", outputCol = "stock_code_encoded"),
          #Country variable
          StringIndexer(inputCol='Country', outputCol='country_index', handleInvalid='keep'),
          OneHotEncoder(inputCol='country_index', outputCol='country_encoded'),
          #Discretizer in slmall, normal or big spent
          QuantileDiscretizer(numBuckets=3, inputCol='SumTotalBought', outputCol='total_bought_index'),
          #
          VectorAssembler(inputCols=['stock_code_encoded', 'country_encoded', 'total_bought_index'],
                          outputCol='features_raw'),
          #Normalize the data
          Normalizer(inputCol="features_raw", outputCol="features", p=1.0),
          #Clustering, for example: 3 clusters
          KMeans(featuresCol='features').setK(3).setSeed(42)]

pipeline = Pipeline(stages=stages)

model = pipeline.fit(joined_df)

In [15]:
df_with_clusters = model.transform(joined_df).cache()
df_with_clusters.limit(5).toPandas()

,CustomerID,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalBought,SumTotalBought,stock_code_index,stock_code_encoded,country_index,country_encoded,total_bought_index,features_raw,features,prediction
0,17850,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,United Kingdom,15.30,5288.63,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1,17850,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63,403.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
2,17850,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,United Kingdom,22.00,5288.63,452.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
3,17850,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63,288.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
4,17850,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,United Kingdom,20.34,5288.63,281.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [16]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(df_with_clusters)
silhouette

0.3471978832087873